# ATEK Demo 3: Model training and evaluation benchmarking
In this demo, we will show how to run a mini-ML-training of CubeRCNN model for 3D object detection task, using ATEK preprocessed data from ATEK Data Store. We will also show examples of how to run evaluation benchmarking 


In [ ]:
import os
import logging
import sys

# Configure logging to display the log messages in the notebook
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

logger = logging.getLogger()

# Set up code and data paths
atek_src_path = os.path.join(os.path.expanduser("~"), "atek_on_fbsource")
workdir = "/home/louy/Calibration_data_link/Atek/2024_08_05_DryRun"
train_data_dir = os.path.join(workdir, "downloaded_local_wds")


## Example 1: mini-CubeRCNN training with ATEK data
First, follow Demo 2 to download ATEK preprocessed data. Then user can call our `tools/train_cubercnn.py` script to do a mini-scale training. 

Core code snippets in the script (check out the script for full details): 
```
tar_file_urls = load_yaml_and_extract_paths(train_list_yaml)
data_loader = create_atek_dataloader_as_cubercnn(urls = tar_file_urls, ...)
# Loop over cubercnn-format data samples
for sample_data in data_loader:
    loss = model(sample_data)
    # perform optimization
    ...
```

For this mini-training example, we will run in terminal instead of in Python notebook. 

In [ ]:
mini_training_command = (
  f"mamba activate atek; "
  f"python {atek_src_path}/tools/train_cubercnn.py"
  f" --config-file {workdir}/cubercnn_train_config_mini_example.yaml"
  f" --num-gpus 1"
  f" OUTPUT_DIR {workdir}/mini_test_1"
  f" TRAIN_LIST {train_data_dir}/local_train_tars.yaml"
  f" TEST_LIST {train_data_dir}/local_validation_tars.yaml" 
  f" CATEGORY_JSON {atek_src_path}/data/atek_id_to_name.json" 
  f" ID_MAP_JSON {atek_src_path}/data/atek_name_to_id.json" 
  f" MODEL.WEIGHTS_PRETRAIN /home/louy/Calibration_data_link/Atek/cubercnn_DLA34_FPN.pth"
)
logger.info(f"Please run the following command in terminal to perform mini-scale training: \n {mini_training_command}\n")

# Inspect training progress using tensorboard
tensorboard_command = "mamba activate atek; tensorboard --logdir=./ --port 6007 --samples_per_plugin=images=1000"
logger.info(f"You can run the following command to open a tensorboard to check training progress: \n {tensorboard_command}\n")

## Example 2: Run benchmarking on 3D Object Detection Task
Another feature provided in ATEK is per-ML-task benchmarking scripts to evaluate model performance. Currently we support 2 ML tasks in benchmakring: 
1. Static 3D object detection.
2. 3D surface reconstruction.

Here we will demonstrate how to run static 3D object detection benchmarking `tools/benchmarking_static_object_detction.py`. The script expects provide 2 csv files as input, groundtruth and predictions, both in the same predefined ATEK format (see below). The benchmarking script will compute and report a number of detection metrics. 


In [ ]:
benchmarking_command = (
    f"mamba activate atek;"
    f"python {atek_src_path}/tools/benchmarking_static_object_detection.py "
    f" --pred-csv {workdir}/eval_results/prediction_obbs.csv"
    f" --gt-csv {workdir}/eval_results/gt_obbs.csv"
    f" --output-file {workdir}/eval_results/atek_metrics.json"
)
logger.info(f"Please run the following command in terminal to perform 3D obb benchmarking: \n {benchmarking_command}\n")